# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need


# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.createOrReplaceTempView("user_log_table")

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [6]:
# TODO: write your code to answer question 1

In [7]:
spark.sql('''
          SELECT DISTINCT page
          FROM user_log_table 
          WHERE userId == ""
          '''
          ).show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



These are the pages `userId == ""`visited. 

In [8]:
spark.sql('''
          SELECT DISTINCT page FROM user_log_table
          '''
         ).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



These are all the pages present. So we have to perform a right join to see all the pages userId == "" did not visit

In [9]:
spark.sql('''
         SELECT * FROM
          (SELECT DISTINCT page
          FROM user_log_table 
          WHERE userId == "") AS user_pages
          RIGHT JOIN (SELECT DISTINCT page FROM user_log_table) AS all_pages
          ON user_pages.page =  all_pages.page
          '''
          ).show()

+-----+----------------+
| page|            page|
+-----+----------------+
| null|Submit Downgrade|
| Home|            Home|
| null|       Downgrade|
| null|          Logout|
| null|   Save Settings|
|About|           About|
| null|        Settings|
|Login|           Login|
| null|        NextSong|
| Help|            Help|
| null|         Upgrade|
| null|           Error|
| null|  Submit Upgrade|
+-----+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

Prefer Spark SQL:

- **SQL Expertise**: its easier

- **Existing SQL Codebase**: If you have existing SQL code or are transitioning from a traditional relational database environment, using Spark SQL can make it easier to reuse and adapt your SQL queries for big data processing.


Prefer Spark DataFrames:

- **Expressiveness and Flexibility**: Spark DataFrames provide a more expressive and flexible API for data manipulation using a syntax similar to pandas or R data frames. This is especially useful for complex data transformations and analysis.


- **Performance**: While Spark SQL and DataFrames are closely integrated, in some cases, expressing your logic using the DataFrame API directly may provide more control and better performance for certain operations.


# Question 3

How many female users do we have in the data set?

In [ ]:
# TODO: write your code to answer question 3

In [31]:
spark.sql('''
         SELECT 
             distinct userId,
             gender
         FROM user_log_table
          '''
          ).show()

+------+------+
|userId|gender|
+------+------+
|  2902|     F|
|  1872|     M|
|   838|     F|
|   314|     M|
|  1941|     M|
|  1545|     M|
|   577|     F|
|  2122|     F|
|  2049|     M|
|  2967|     M|
|  1887|     F|
|  1661|     F|
|  1403|     M|
|  2546|     F|
|  1866|     F|
|  1162|     F|
|  1232|     M|
|  2675|     F|
|  1865|     M|
|  2933|     M|
+------+------+
only showing top 20 rows



In [34]:
spark.sql('''
          SELECT distinct userId
          FROM user_log_table
          '''
          ).show()

+------+
|userId|
+------+
|  2904|
|   691|
|  2294|
|  2162|
|  1436|
|  2088|
|  2275|
|  2756|
|   800|
|  1394|
|   451|
|   926|
|  1746|
|  2696|
|  1280|
|   870|
|     7|
|  1903|
|   591|
|   613|
+------+
only showing top 20 rows



In [40]:
spark.sql('''
         SELECT count(gender)
         FROM (SELECT distinct userId, gender FROM user_log_table)
         GROUP BY gender
         '''
          ).show()

+-------------+
|count(gender)|
+-------------+
|          462|
|            0|
|          501|
+-------------+



# Question 4

How many songs were played from the most played artist?

In [41]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [50]:
# TODO: write your code to answer question 4

spark.sql('''
         SELECT artist, count(artist)
         FROM user_log_table
         GROUP BY artist
         ORDER BY count(song) DESC
          '''
          ).show()

+--------------------+-------------+
|              artist|count(artist)|
+--------------------+-------------+
|            Coldplay|           83|
|       Kings Of Leon|           69|
|Florence + The Ma...|           52|
|            BjÃÂ¶rk|           46|
|       Dwight Yoakam|           45|
|       Justin Bieber|           43|
|      The Black Keys|           40|
|         OneRepublic|           37|
|                Muse|           36|
|        Jack Johnson|           36|
|           Radiohead|           31|
|        Taylor Swift|           29|
|               Train|           28|
|Barry Tuckwell/Ac...|           28|
|          Lily Allen|           28|
|          Nickelback|           27|
|           Metallica|           27|
|           Daft Punk|           27|
|          Kanye West|           26|
|Red Hot Chili Pep...|           24|
+--------------------+-------------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

Was not able to solve it, looked at the solutions.

In [69]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql(
            '''
            SELECT 
                userID, 
                page, 
                ts, 
                CASE 
                    WHEN page = 'Home' 
                        THEN 1
                        ELSE 0 
                END AS is_home 
            FROM user_log_table 
            WHERE (page = 'NextSong') or (page = 'Home') 
            ''')


# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

In [70]:
is_home.show()

+------+--------+-------------+-------+
|userID|    page|           ts|is_home|
+------+--------+-------------+-------+
|  1046|NextSong|1513720872284|      0|
|  1000|NextSong|1513720878284|      0|
|  2219|NextSong|1513720881284|      0|
|  2373|NextSong|1513720905284|      0|
|  1747|    Home|1513720913284|      1|
|  1162|NextSong|1513720955284|      0|
|  1061|NextSong|1513720959284|      0|
|   748|    Home|1513720959284|      1|
|   597|    Home|1513720980284|      1|
|  1806|NextSong|1513720983284|      0|
|   748|NextSong|1513720993284|      0|
|  1176|NextSong|1513721031284|      0|
|  2164|NextSong|1513721045284|      0|
|  2146|NextSong|1513721058284|      0|
|  2219|NextSong|1513721077284|      0|
|  1176|    Home|1513721088284|      1|
|  2904|NextSong|1513721095284|      0|
|   597|NextSong|1513721097284|      0|
|   226|NextSong|1513721104284|      0|
|  1046|NextSong|1513721104284|      0|
+------+--------+-------------+-------+
only showing top 20 rows



In [77]:
# find the cumulative sum over the is_home column
cumulative_sum = spark.sql(
        '''
        SELECT *, SUM(is_home) OVER
        (PARTITION BY userID 
        ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
        FROM is_home_table
        '''
        )


# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

In [78]:
# find the average count for NextSong
spark.sql('''
          SELECT AVG(count_results) FROM 
          (SELECT COUNT(*) AS count_results FROM period_table 
          GROUP BY userID, period, page HAVING page = 'NextSong') AS counts
''').show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

